Authors:
- Abdulaziz Alakooz : Github([@3koozy](https://github.com/3koozy)).
- Ahad Algrais : Github([@ahadalgrais](https://github.com/ahadalgrais)).
- Mujtaba Alghadeer : Github([@ghadeem](https://github.com/ghadeem)).

In [ ]:
#Prequisites:
!pip install rasterio
!pip install radiant_mlhub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 63.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1
  Attempting unins

In [ ]:
#Import needed libraries:
import os
import glob
import json
import getpass
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
from radiant_mlhub import Dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.base import is_classifier
from sklearn.metrics import accuracy_score

In [ ]:
os.environ['MLHUB_API_KEY'] = 'f31d9911aff9a7ba88e4be73b25a41f960393cf31c5745da97b7f54bf31a1f62'

# AgriFieldNet Dataset / Competition

[link to compeition and dataset description](https://zindi.africa/competitions/agrifieldnet-india-challenge)

Small farms (<2ha) produce about 35% of the world’s food, and are mostly found in low- and middle-income countries. Reliable information about these farms is limited, making support and policy-making difficult. Earth Observation data from satellites such as Sentinel-2, in combination with machine learning, can help improve agricultural monitoring, crop mapping, and disaster risk management for these small farms.

The label chips contain the mapping of pixels to crop type labels. The following pixel values correspond to the following crop types:

* 1 - Wheat
* 2 - Mustard
* 3 - Lentil
* 4 - No Crop/Fallow
* 5 - Green pea
* 6 - Sugarcane
* 8 - Garlic
* 9 - Maize
* 13 - Gram
* 14 - Coriander
* 15 - Potato
* 16 - Bersem
* 36 - Rice

for some models we need to re-label these classes in ascending order without any gaps.

**Note:** this notebook has been heavily inspired by this starter notebook ([link here](https://github.com/radiantearth/agrifieldnet_india_competition/blob/main/Starter%20notebook.ipynb)).

## Prepare the dataset:

In [ ]:
# Select desired bands 

all_bands = ['B01', 'B02', 'B03', 'B04','B05', 'B06', 'B07', 'B08','B8A', 'B09', 'B11', 'B12']

selected_bands = all_bands#all_bands[1:4]  + [all_bands[7]] 
selected_bands

['B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B8A',
 'B09',
 'B11',
 'B12']

In [ ]:
# Define the dataset collection_id, desired assets, and collections

main = 'ref_agrifieldnet_competition_v1'

assets = ['field_ids','raster_labels']

source_collection = f'{main}_source'
train_label_collection = f'{main}_labels_train'
test_label_collection = f'{main}_labels_test'

dataset = Dataset.fetch(main)

my_filter = dict(
    ref_agrifieldnet_competition_v1_labels_train=assets,

    ref_agrifieldnet_competition_v1_labels_test=[assets[0]],

    ref_agrifieldnet_competition_v1_source=selected_bands 
)

dataset.download(collection_filter=my_filter)

ref_agrifieldnet_competition_v1: fetch stac catalog: 1044KB [00:01, 679.90KB/s]                          
unarchive ref_agrifieldnet_competition_v1.tar.gz: 100%|██████████| 6186/6186 [00:01<00:00, 4541.93it/s]
filter by collection ids and asset keys: 193370it [00:00, 1807855.00it/s]         
download assets: 100%|██████████| 17643/17643 [36:50<00:00,  7.98it/s]


In [ ]:
#load collection json and retrieve all unique folder ids 
#use all unique folder ids to create a list of field and label paths for all tiles

with open (f'{main}/{train_label_collection}/collection.json') as f:
    train_json = json.load(f)
    
train_folder_ids = [i['href'].split('_')[-1].split('.')[0] for i in train_json['links'][4:]]
del train_folder_ids[-1]

train_field_paths = [f'{main}/{train_label_collection}/{train_label_collection}_{i}/field_ids.tif' for i in train_folder_ids]
train_label_paths = [f'{main}/{train_label_collection}/{train_label_collection}_{i}/raster_labels.tif' for i in train_folder_ids]

In [ ]:
#create dataset for folder_ids and field_paths

competition_train_data = pd.DataFrame(train_folder_ids, columns=['unique_folder_id'])
competition_train_data['field_paths'] = train_field_paths
competition_train_data.head()

,unique_folder_id,field_paths
0,28852,ref_agrifieldnet_competition_v1/ref_agrifieldn...
1,d987c,ref_agrifieldnet_competition_v1/ref_agrifieldn...
2,ca1d4,ref_agrifieldnet_competition_v1/ref_agrifieldn...
3,2ec18,ref_agrifieldnet_competition_v1/ref_agrifieldn...
4,7575d,ref_agrifieldnet_competition_v1/ref_agrifieldn...


In [ ]:
#Extract field_crop Pairs 

def field_crop_extractor(crop_field_files):
    field_crops = {}

    for label_field_file in tqdm(crop_field_files):
        with rasterio.open(f'{main}/{train_label_collection}/{train_label_collection}_{label_field_file}/field_ids.tif') as src:
            field_data = src.read()[0]
        with rasterio.open(f'{main}/{train_label_collection}/{train_label_collection}_{label_field_file}/raster_labels.tif') as src:
            crop_data = src.read()[0]
    
        for x in range(0, crop_data.shape[0]):
            for y in range(0, crop_data.shape[1]):
                field_id = str(field_data[x][y])
                field_crop = crop_data[x][y]

                if field_crops.get(field_id) is None:
                    field_crops[field_id] = []

                if field_crop not in field_crops[field_id]:
                    field_crops[field_id].append(field_crop)
    
    field_crop_map  =[[k, v[0]]  for k, v in field_crops.items() ]
    field_crop = pd.DataFrame(field_crop_map , columns=['field_id','crop_id'])

    return field_crop[field_crop['field_id']!='0']

In [ ]:
field_crop_pair = field_crop_extractor(train_folder_ids)
field_crop_pair.head()

100%|██████████| 1165/1165 [02:04<00:00,  9.32it/s]


,field_id,crop_id
1,757,6
2,756,6
3,1372,5
4,1374,1
5,1986,4


In [ ]:
field_crop_pair.shape

(5551, 2)

In [ ]:
# Our goal is developing a pixel-based Random Forest model. So we will create an X variable
# such that, each row is a pixel and each column is one of the band observations mapped to its corresponding field. 


img_sh = 256
n_selected_bands= len(selected_bands)

n_obs = 1  #imagery per chip(no time series)

def feature_extractor(data_ ,   path ):
    '''
        data_: Dataframe with 'field_paths' and 'unique_folder_id' columns
        path: Path to source collections files

        returns: pixel dataframe with corresponding field_ids
        '''
    
    X = np.empty((0, n_selected_bands * n_obs))
    X_tile = np.empty((img_sh * img_sh, 0))
    X_arrays = []
        
    field_ids = np.empty((0, 1))

    for idx, tile_id in tqdm(enumerate(data_['unique_folder_id'])):
        
        field_src =   rasterio.open( data_['field_paths'].values[idx])
        field_array = field_src.read(1)
        field_ids = np.append(field_ids, field_array.flatten())
        
        
        bands_src = [rasterio.open(f'{main}/{path}/{path}_{tile_id}/{band}.tif') for band in selected_bands]
        bands_array = [np.expand_dims(band.read(1).flatten(), axis=1) for band in bands_src]
        
        X_tile = np.hstack(bands_array)

        X_arrays.append(X_tile)
        

    X = np.concatenate(X_arrays)
    
    data = pd.DataFrame(X, columns=selected_bands)

    data['field_id'] = field_ids

    return data[data['field_id']!=0]

In [ ]:
# competition_train_data.drop(1165, inplace=True)

In [ ]:
train_data = feature_extractor(competition_train_data, source_collection)
train_data.head()

1165it [03:44,  5.19it/s]


,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12,field_id
11031,43,39,38,38,41,54,63,61,64,12,57,37,757.0
11287,43,39,38,38,42,57,67,63,72,12,63,42,757.0
11288,43,39,38,37,41,59,69,65,78,12,68,43,757.0
11289,43,38,37,36,41,59,69,64,78,12,68,43,757.0
11543,43,39,38,38,42,57,67,64,72,12,63,42,757.0


In [ ]:
# The following are the derived indices:

# NDVI (Normalized Green Red Difference Index) : (B08 - B04)/ (B08 + B04)
# GLI (Green Leaf Index) : (2 * B03 - B04 - B02)/(2 * B03 + B04 + B02)
# CVI : (Chlorophyll Vegetation Index) : (B08 / B03) * (B04 / B03)
# SIPI : (B08 - B02) / (B08 - B04)
# S2REP : 705 + 35 * (((( B07 + B04 ) / 2) - B05 ) / (B06 - B05))
# CCCI : ((B08 - B05) / (B08 + B05)) / ((B08 - B04) / (B08 + B04))
# HUE (Overall Hue Index) : atan( 2 * ( B02 - B03 - B04 ) / 30.5 * ( B03 - B04 ))
# RENDVI : (B06 - B05) / (B06 + B05)
# RECI (Chlorophyll Index) : ( B08 / B04 ) - 1
# EVI (Enhanced Vegetation Index) : (2.5 * (B08 - B04) / ((B08 + 6.0 * B04 - 7.5 * B02) + 1.0))
# EVI2 (Enhanced Vegetation Index 2) : (2.4 * (B08 - B04) / (B08 + B04 + 1.0))
# NDWI : (B04 - B02) / (B04 + B02)
# NPCRI : (B03 - B08) / (B03 + B08)

In [ ]:
import math

#add pre-processing features:
def add_preprocessing_features(train_data):
  train_data["NDVI"] = (train_data["B08"]-train_data["B04"]) / (train_data["B08"] + train_data["B04"])
  train_data["GLI"] = (2 * train_data.B03 - train_data.B04 - train_data.B02)/(2 * train_data.B03 + train_data.B04 + train_data.B02)
  train_data["CVI"] =  (train_data.B08 / train_data.B03) * (train_data.B04 / train_data.B03)
  train_data["SIPI"] = (train_data.B08 - train_data.B02) / (train_data.B08 - train_data.B04)
  train_data["S2REP"] = 705 + 35 * (((( train_data.B07 + train_data.B04 ) / 2) - train_data.B05 ) / (train_data.B06 - train_data.B05))
  train_data["CCCI"] = ((train_data.B08 - train_data.B05) / (train_data.B08 + train_data.B05)) / ((train_data.B08 - train_data.B04) / (train_data.B08 + train_data.B04))
  train_data["HUE"] = np.arctan( 2 * ( train_data.B02 - train_data.B03 - train_data.B04 ) / 30.5 * ( train_data.B03 - train_data.B04 ))
  train_data["RENDVI"] = (train_data.B06 - train_data.B05) / (train_data.B06 + train_data.B05)
  train_data["RECI"] = ( train_data.B08 / train_data.B04 ) - 1
  train_data["EVI"] = (2.5 * (train_data.B08 - train_data.B04) / ((train_data.B08 + 6.0 * train_data.B04 - 7.5 * train_data.B02) + 1.0))
  train_data["EVI2"] = (2.4 * (train_data.B08 - train_data.B04) / (train_data.B08 + train_data.B04 + 1.0))
  train_data["NDWI"] = (train_data.B04 - train_data.B02) / (train_data.B04 + train_data.B02)
  train_data["NPCRI"] = (train_data.B03 - train_data.B08) / (train_data.B03 + train_data.B08)

  #fille nan with mean:
  S2REP = train_data.S2REP.mean()
  EVI = train_data.EVI.mean()

  train_data.S2REP.fillna(value=S2REP, inplace=True)
  train_data.EVI.fillna(value=EVI, inplace=True)
  train_data.replace([np.nan], 0, inplace=True)
  train_data.dropna(inplace=True)

  #replace inf with max 0:
  train_data.replace([np.inf, -np.inf], 0, inplace=True)

add_preprocessing_features(train_data)
train_data.head()

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,...,SIPI,S2REP,CCCI,HUE,RENDVI,RECI,EVI,EVI2,NDWI,NPCRI
11031,43,39,38,38,41,54,63,61,64,12,...,0.956522,730.576923,0.843990,0.000000,0.136842,0.605263,-23.000000,0.552000,3.311688,2.353535
11287,43,39,38,38,42,57,67,63,72,12,...,0.960000,729.500000,0.808000,0.000000,0.151515,0.657895,-125.000000,0.588235,3.311688,2.287129
11288,43,39,38,37,41,59,69,65,78,12,...,0.928571,728.333333,0.824798,1.406529,0.180000,0.756757,-15.555556,0.652427,3.342105,2.223301
11289,43,38,37,36,41,59,69,64,78,12,...,0.928571,727.361111,0.782313,1.408264,0.180000,0.777778,-17.500000,0.665347,3.432432,2.267327
11543,43,39,38,38,42,57,67,64,72,12,...,0.961538,729.500000,0.814224,0.000000,0.151515,0.684211,130.000000,0.605825,3.311688,2.254902


In [ ]:
from sklearn.preprocessing import LabelEncoder

def get_dataset(df, field_crop_pair, stats='mean'):
  dataset_df = None

  if stats=='mean':
    dataset_df = df.groupby(['field_id']).mean().reset_index()
    dataset_df.field_id = [str(int(i)) for i in dataset_df.field_id.values]
  elif stats=='median':
    dataset_df = df.groupby(['field_id']).median().reset_index()
    dataset_df.field_id = [str(int(i)) for i in dataset_df.field_id.values]
  elif stats=='std':
    dataset_df = df.groupby(['field_id']).std().reset_index()
    dataset_df.field_id = [str(int(i)) for i in dataset_df.field_id.values]
  elif stats=='variance':
    dataset_df = df.groupby(['field_id']).var().reset_index()
    dataset_df.field_id = [str(int(i)) for i in dataset_df.field_id.values]
  elif stats=='min':
    dataset_df = df.groupby(['field_id']).min().reset_index()
    dataset_df.field_id = [str(int(i)) for i in dataset_df.field_id.values]
  elif stats=='max':
    dataset_df = df.groupby(['field_id']).max().reset_index()
    dataset_df.field_id = [str(int(i)) for i in dataset_df.field_id.values]
  elif stats=='mode':
    dataset_df = df.groupby(['field_id']).agg(lambda x:x.value_counts().index[0]).reset_index()
    dataset_df.field_id = [str(int(i)) for i in dataset_df.field_id.values]
  elif stats=='skew':
    dataset_df = df.groupby(['field_id']).skew().reset_index()
    dataset_df.field_id = [str(int(i)) for i in dataset_df.field_id.values]
  elif stats=='kurt':
    dataset_df = df.groupby(['field_id']).agg(pd.Series.kurt).reset_index()
    dataset_df.field_id = [str(int(i)) for i in dataset_df.field_id.values]
  
  if not isinstance(field_crop_pair, type(None)):
    train_df = pd.merge(dataset_df, field_crop_pair , on='field_id' )
  else : train_df = dataset_df

  #check for NaN:
  train_df.replace([np.nan], 0, inplace=True)

  return train_df

def get_dataset_np(dataset_df, label_col='crop_id', label_encoder=True):
  x = dataset_df.drop([label_col, 'field_id'], axis=1) if label_col in dataset_df.columns else dataset_df.drop("field_id", axis=1)
  y = dataset_df[label_col] if label_col in dataset_df.columns else None

  if label_encoder and  not isinstance(y, type(None)):
    le = LabelEncoder()
    y = le.fit_transform(y)

  return x,y.astype(int) if not isinstance(y, type(None)) else None

In [ ]:
stats = ['mean', 'median', 'std', 'min', 'max', 'mode', 'skew', 'kurt']
datasets = [get_dataset(train_data, field_crop_pair, stat) for stat in stats]

#Prepare datasets:
datasets = [get_dataset_np(ds) for ds in datasets]

# Perform train-test split for each dataset
train_test_splits = [train_test_split(X, y, test_size=0.2, random_state=42) for X, y in datasets]
X_train_list, X_test_list, y_train_list, y_test_list = zip(*train_test_splits)

## Model Training:

In [ ]:
random_state = 1234

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

#this model allows a combination of multiple sklearn models to be trained on uniques datasets:
class ModelEnsemble(BaseEstimator, ClassifierMixin):
    def __init__(self, models):
        self.models = models
        self.weights = None

    #takes a list of qunique dataset per model:
    def fit(self, X_list, y_list):
        for model, X, y in zip(self.models, X_list, y_list):
            model.fit(X, y)

    def predict(self, X_list):
        predictions = np.array([model.predict(X) for model, X in zip(self.models, X_list)])
        ensemble_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions)
        return ensemble_predictions

    def predict_prob(self, X_list):
        prob_list = np.array([model.predict_proba(X) for model, X in zip(self.models, X_list)])
        if self.weights is not None:
            prob_list = prob_list * np.array(self.weights)[:, np.newaxis, np.newaxis]
        ensemble_prob = prob_list.sum(axis=0) / prob_list.sum(axis=0).sum(axis=1, keepdims=True)
        return ensemble_prob

    #define each model wight/contribution of the final prediction score based on its f1 score:
    def calculate_weights(self, X_test_list, y_test):
        f1_scores = np.array([f1_score(y_test, model.predict(X_test), average='weighted') for model, X_test in zip(self.models, X_test_list)])
        self.weights = f1_scores / f1_scores.sum()

    def model_f1_scores(self, X_test_list, y_test):
        f1_scores = np.array([f1_score(y_test, model.predict(X_test), average='weighted') for model, X_test in zip(self.models, X_test_list)])
        return f1_scores

### Random Forests Model ensemble:

In [ ]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone

foundation_model = RandomForestClassifier(n_estimators = 20, random_state = 0, n_jobs = 3)

#train multiple random forest models:
models = [clone(foundation_model) for i in range(len(datasets))]

# Create an ensemble of the RandomForest models
model = ModelEnsemble(models=models)
model_type = 'ensemble'

# Fit the ensemble model
model.fit(X_train_list, y_train_list)

# Calculate weights based on F1 scores
model.calculate_weights(X_test_list, y_test_list[0])
print("weights : ",model.weights)

#show indivisual model performence:
print("Indivisual model performence : ", model.model_f1_scores(X_test_list,y_test_list[0]))

# Make predictions
predictions = model.predict(X_test_list)
print("Ensemble accuracy:", accuracy_score(y_test_list[0], predictions))

# Get class probabilities
probabilities = model.predict_prob(X_test_list)
print("Ensemble probabilities:\n", probabilities)

KeyboardInterrupt: ignored

### XGBoost Model Ensemble:

In [ ]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from xgboost import XGBClassifier

foundation_model = XGBClassifier(eta= 0.05, max_depth= 5, objective= "multi:softprob", num_class=13, n_estimators=400, learning_rate= 0.1)

#train multiple random forest models:
models = [clone(foundation_model) for i in range(len(datasets))]

# Create an ensemble of the RandomForest models
model = ModelEnsemble(models=models)
model_type = 'ensemble'

# Fit the ensemble model
model.fit(X_train_list, y_train_list)

# Calculate weights based on F1 scores
model.calculate_weights(X_test_list, y_test_list[0])
print("weights : ",model.weights)

#show indivisual model performence:
print("Indivisual model performence : ", model.model_f1_scores(X_test_list,y_test_list[0]))

# Make predictions
predictions = model.predict(X_test_list)
print("Ensemble accuracy:", accuracy_score(y_test_list[0], predictions))

# Get class probabilities
probabilities = model.predict_prob(X_test_list)
print("Ensemble probabilities:\n", probabilities)

### Neural Network Model Ensemble:

In [ ]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.neural_network import MLPClassifier

#define a foundation model to be cloned and used as a start point by all models in the ensemble:
foundation_model = MLPClassifier(hidden_layer_sizes=(256,128,64,32), activation='relu', solver='adam', max_iter=200, learning_rate_init=0.01, early_stopping=False , verbose=True, random_state=random_state)

#train multiple random forest models:
models = [clone(foundation_model) for i in range(len(datasets))]

# Create an ensemble of the RandomForest models
model = ModelEnsemble(models=models)
model_type = 'ensemble'

# Fit the ensemble model
model.fit(X_train_list, y_train_list)

# Calculate weights based on F1 scores
model.calculate_weights(X_test_list, y_test_list[0])
print("weights : ",model.weights)

#show indivisual model performence:
print("Indivisual model performence : ", model.model_f1_scores(X_test_list,y_test_list[0]))

# Make predictions
predictions = model.predict(X_test_list)
print("Ensemble accuracy:", accuracy_score(y_test_list[0], predictions))

# Get class probabilities
probabilities = model.predict_prob(X_test_list)
print("Ensemble probabilities:\n", probabilities)

Iteration 1, loss = 11.40332129
Iteration 2, loss = 1.89449359
Iteration 3, loss = 1.67269075
Iteration 4, loss = 1.65130135
Iteration 5, loss = 1.64879586
Iteration 6, loss = 1.63237996
Iteration 7, loss = 1.58243207
Iteration 8, loss = 1.54733337
Iteration 9, loss = 1.53390050
Iteration 10, loss = 1.52198405
Iteration 11, loss = 1.51290923
Iteration 12, loss = 1.54199123
Iteration 13, loss = 1.51237127
Iteration 14, loss = 1.49815334
Iteration 15, loss = 1.48478364
Iteration 16, loss = 1.50132050
Iteration 17, loss = 1.47757940
Iteration 18, loss = 1.46437787
Iteration 19, loss = 1.44893579
Iteration 20, loss = 1.42451913
Iteration 21, loss = 1.39989042
Iteration 22, loss = 1.41905747
Iteration 23, loss = 1.39199061
Iteration 24, loss = 1.39840420
Iteration 25, loss = 1.35125386
Iteration 26, loss = 1.32629526
Iteration 27, loss = 1.32155045
Iteration 28, loss = 1.31538084
Iteration 29, loss = 1.31187335
Iteration 30, loss = 1.28881461
Iteration 31, loss = 1.25209150
Iteration 32, lo

### Single Neural network Model:

In [ ]:
#train single NN model on mean stats of train dataset:
le = LabelEncoder()
y_train = le.fit_transform(y_train)

random_state=1234
model = MLPClassifier(hidden_layer_sizes=(256,128,64,32), activation='relu', solver='adam', max_iter=200, learning_rate_init=0.01, early_stopping=False , verbose=True, random_state=random_state)#, batch_size=32, learning_rate='adaptive')
model_type = 'single'

model.fit(X_train_list[0], y_train_list[0].astype(int))

## Model Evaluation:

In [ ]:
if model_type == 'ensemble':
  y_pred_crop = model.predict_prob(X_test_list)
  y_pred_crop = y_pred_crop.argmax(axis=1)
  print(classification_report(y_test_list[0],y_pred_crop))
elif model_type == 'single':
  y_pred_crop =  model.predict(X_test_list[0])
  print(classification_report(y_test_list[0],y_pred_crop))

              precision    recall  f1-score   support

           0       0.59      0.83      0.69       400
           1       0.35      0.09      0.14       181
           2       0.00      0.00      0.00        25
           3       0.68      0.81      0.74       351
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00        39
           6       0.00      0.00      0.00        12
           7       0.67      0.89      0.76        65
           8       0.00      0.00      0.00         9
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00        15

    accuracy                           0.62      1111
   macro avg       0.18      0.20      0.18      1111
weighted avg       0.52      0.62      0.55      1111



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Feature Importance:

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.base import is_classifier
from sklearn.metrics import accuracy_score

def softmax_stable(x):
    return(np.exp(x - np.max(x)) / np.exp(x - np.max(x)).sum())

def permutation_importance(model, X, y, n_iter=5, random_state=None, scoring=None):
    if scoring is None:
        scoring = accuracy_score if is_classifier(model) else mean_absolute_error

    np.random.seed(random_state)

    baseline_score = scoring(y, model.predict(X))
    feature_importances = np.zeros(X.shape[1])

    for feature_idx in range(X.shape[1]):
        scores = np.zeros(n_iter)
        for iter_idx in range(n_iter):
            X_permuted = X.copy()
            np.random.shuffle(X_permuted[:, feature_idx])
            permuted_score = scoring(y, model.predict(X_permuted))
            scores[iter_idx] = permuted_score - baseline_score
        feature_importances[feature_idx] = np.mean(scores)

    return feature_importances

In [ ]:
# Calculate permutation importance
if model_type == 'ensemble':
  importances = permutation_importance(model.models[0], X_test_list[0].to_numpy(), y_test_list[0], n_iter=5, random_state=42)
else:
  importances = permutation_importance(model, X_test_list[0].to_numpy(), y_test_list[0], n_iter=5, random_state=42)

# Print feature importances
print("Permutation importance of each feature:\n", importances)
print("Permutation importance of each feature (softmax):\n", softmax_stable(importances))

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifi

Permutation importance of each feature:
 [-0.00018002 -0.00252025 -0.0090009  -0.050045   -0.01026103 -0.00288029
 -0.01026103 -0.00432043 -0.01116112  0.0039604  -0.16867687 -0.0129613
  0.00108011  0.00018002 -0.00054005  0.00252025 -0.00090009  0.00108011
 -0.00072007  0.         -0.00108011  0.00378038  0.00162016 -0.00108011
  0.00054005]
Permutation importance of each feature (softmax):
 [0.04040788 0.04031343 0.04005301 0.03844235 0.04000257 0.04029892
 0.04000257 0.04024092 0.03996658 0.04057553 0.03414199 0.0398947
 0.04045883 0.04042243 0.04039333 0.04051714 0.04037879 0.04045883
 0.04038606 0.04041515 0.04037153 0.04056823 0.04048069 0.04037153
 0.04043699]


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


## Prepare and Submit test data results:

In [ ]:
with open (f'{main}/{test_label_collection}/collection.json') as f:
    test_json = json.load(f)
    
test_folder_ids = [i['href'].split('_')[-1].split('.')[0] for i in test_json['links'][4:]]

test_field_paths = [f'{main}/{test_label_collection}/{test_label_collection}_{i}/field_ids.tif' for i in test_folder_ids]

In [ ]:
competition_test_data = pd.DataFrame(test_folder_ids , columns=['unique_folder_id'])
competition_test_data['field_paths'] = test_field_paths
competition_test_data.drop(707, inplace=True)
competition_test_data.head()

,unique_folder_id,field_paths
0,6199c,ref_agrifieldnet_competition_v1/ref_agrifieldn...
1,6c81d,ref_agrifieldnet_competition_v1/ref_agrifieldn...
2,1ebeb,ref_agrifieldnet_competition_v1/ref_agrifieldn...
3,586a2,ref_agrifieldnet_competition_v1/ref_agrifieldn...
4,65812,ref_agrifieldnet_competition_v1/ref_agrifieldn...


In [ ]:
test_data = feature_extractor(competition_test_data,  source_collection)
test_data.head()

707it [01:29,  7.89it/s]


,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12,field_id
35283,39,35,35,35,38,48,55,59,60,11,53,39,5407.0
35284,39,34,33,34,37,49,58,58,63,11,54,40,5407.0
35538,39,36,36,37,39,59,70,56,76,14,55,37,5407.0
35539,39,35,36,34,39,59,70,75,76,14,55,37,5407.0
35540,39,33,34,31,37,70,85,79,90,14,54,34,5407.0


In [ ]:
add_preprocessing_features(test_data)
test_data.head()

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,...,SIPI,S2REP,CCCI,HUE,RENDVI,RECI,EVI,EVI2,NDWI,NPCRI
35283,39,35,35,35,38,48,55,59,60,11,...,1.000000,729.500000,0.847938,0.000000,0.116279,0.685714,8.000000,0.606316,0.000000,2.468085
35284,39,34,33,34,37,49,58,58,63,11,...,1.000000,731.250000,0.847368,1.570167,0.139535,0.705882,7.500000,0.619355,0.000000,2.538462
35538,39,36,36,37,39,59,70,56,76,14,...,1.052632,730.375000,0.875900,1.570139,0.204082,0.513514,5.277778,0.485106,0.013699,2.565217
35539,39,35,36,34,39,59,70,75,76,14,...,0.975610,727.750000,0.839538,1.488990,0.204082,1.205882,5.857143,0.894545,3.695652,1.954955
35540,39,33,34,31,37,70,85,79,90,14,...,0.958333,727.272727,0.829741,1.517894,0.308411,1.548387,6.486486,1.037838,3.968750,1.867257


In [ ]:
stats = ['mean', 'median', 'std', 'min', 'max', 'mode', 'skew', 'kurt']
datasets = [get_dataset(test_data, None, stat) for stat in stats]
Field_ids = datasets[0].field_id

#Prepare datasets:
datasets = [get_dataset_np(ds) for ds in datasets]

# Perform train-test split for each dataset
X_list, _ = zip(*datasets)

In [ ]:
#re-labeled classes:
crop_dict = {
 0: 'Wheat',
 1: 'Mustac',
 2: 'Lentil',
 3: 'No crop/Fallow',
 4: 'Green pea',
 5: 'Sugarcane',
 6: 'Garlic',
 7: 'Maize',
 8: 'Gram',
 9: 'Coriander',
 10: 'Potato',
 11: 'Bersem',
 12: 'Rice'}

In [ ]:
def labeler(labeled):
    crop_label = np.array([crop_dict.get(f'{int(i)}') for i in labeled])
    return crop_label

In [ ]:
if model_type == 'ensemble':
  predictions = model.predict_prob(X_list)
  crop_columns = [crop_dict.get(i) for i in range(13)]
elif model_type == 'single':
  predictions = model.predict_proba(X_list[0].drop('field_id', axis=1 ))
  crop_columns = [crop_dict.get(i) for i in model.classes_]

test_df  = pd.DataFrame(columns= ['field_id'] + crop_columns)
test_df['field_id'] = Field_ids
test_df[crop_columns]= predictions 
test_df.to_csv('submission.csv', index=False)
test_df.head()

,field_id,Wheat,Mustac,Lentil,No crop/Fallow,Green pea,Sugarcane,Garlic,Maize,Gram,Coriander,Potato,Bersem,Rice
0,11,0.290678,0.231920,0.029039,0.267506,0.007299,0.104186,0.002751,0.013500,0.001682,0.002007,0.012949,0.005366,0.031118
1,13,0.413412,0.125180,0.062089,0.349690,0.005854,0.018281,0.003927,0.001268,0.004147,0.002808,0.002589,0.003930,0.006824
2,19,0.424064,0.224369,0.036572,0.212582,0.006160,0.058677,0.004885,0.004869,0.003366,0.002666,0.006072,0.004908,0.010812
3,21,0.245387,0.158054,0.023952,0.281237,0.007500,0.218110,0.002650,0.015262,0.001836,0.001612,0.012064,0.005001,0.027336
4,25,0.344387,0.106630,0.018291,0.492958,0.003607,0.012806,0.003858,0.000889,0.003572,0.001682,0.001725,0.004306,0.005287
